* Review Data in Table
* Implicit Index for Constraints
* Purpose of Index for Constraints
* Manage Indexes on Database Tables
* Relationship between Table and Indexes
* Indexes for Performance Tuning
* Exercise and Solution - Create Index on order date and order status
  * Index Name: orders_order_date_status_idx
  * Columns: `order_date` and `order_status`
  * Once index is created, make sure to review the explain plans for the below queries.

```sql
SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0';

SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0'
AND order_status = 'COMPLETE';

SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0'
AND order_status = 'CLOSED';
```

* Review Data in Table

```sql
CREATE TABLE users (
	id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	username VARCHAR(50),
	email VARCHAR(50)
);

INSERT INTO users
VALUES
	(1, 'Durga', 'Gadiraju', 'dgadiraju', 'dgadiraju@email.com'),
	(4, 'Scott', 'Tiger', 'stiger', 'stiger@email.com'),
	(2, 'Mickey', 'Mouse', 'mmouse', 'mmouse@email.com'),
	(5, 'Elvis', 'Presley', 'epresley', 'epresley@email.com'),
	(3, 'Charlie', 'Chaplin', 'cchaplin', 'cchaplin@email.com');
	
SELECT ctid, u.* FROM users AS u;

EXPLAIN
SELECT * FROM users
WHERE id = 4;
```

* Overview of Database Object Metadata

Here are the details about Database Object Metadata (using Postgres as reference)
* A Database contains different types of objects - tables, views, stored procedures, etc.
* A Database Table contain columns, data types, constraints, indexes, etc.
* The details about the table such as columns, data types, constraints, indexes, etc is known as metadata.
* The metadata is typically stored in internal schemas or databases. For postgres database tables, the metadata is typically stored in a special schema called as `information_schema`.
* The schema contains tables and views to expose all the metadata related to database tables and other objects.
* Here are some of the important tables/views.
  * `information_schema.tables`
  * `information_schema.constraints`
  * `pg_stat_all_indexes` - not in information_schema

```sql
SELECT * FROM information_schema.tables
WHERE table_schema = 'public';

SELECT * FROM information_schema.columns
WHERE table_schema = 'public' AND table_name = 'users';

SELECT * FROM information_schema.constraint_column_usage
WHERE table_schema = 'public' AND table_name = 'orders';

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';
```

* Implicit Index for Constraints

1. When it comes to database tables, the columns can be defined using different types of constraints. Not null, foreign key, unique, and primary key.
2. Unique and Primary key constraint columns are always indexed.
3. We cannot drop the indexes that are implicitly created for unique and primary key constraints.

```sql
DROP TABLE users;

CREATE TABLE users (
	id INT PRIMARY KEY,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	username VARCHAR(50) UNIQUE,
	email VARCHAR(50)
);

INSERT INTO users
VALUES
	(1, 'Durga', 'Gadiraju', 'dgadiraju', 'dgadiraju@email.com'),
	(4, 'Scott', 'Tiger', 'stiger', 'stiger@email.com'),
	(2, 'Mickey', 'Mouse', 'mmouse', 'mmouse@email.com'),
	(5, 'Elvis', 'Presley', 'epresley', 'epresley@email.com'),
	(3, 'Charlie', 'Chaplin', 'cchaplin', 'cchaplin@email.com');

SELECT * FROM information_schema.constraint_column_usage
WHERE table_schema = 'public' AND table_name = 'users';

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';

DROP INDEX users_pkey; -- The command will fail
```